<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #4: Pre-Procesamiento de textos y Featuring Engineering 
`Fecha de entrega: Marzo 18, 2021. (Antes del inicio de la próxima clase).`

In [1]:
import re
import pandas as pd
from collections import Counter

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from sklearn.feature_extraction.text import CountVectorizer

# Punto 1: Pre-Procesamiento

- `[5 pts]` Leer el archivo `dialogos.csv` usando `pandas`
- `[10 pts]` Crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números y sin palabras vacias
- `[7 pts]` Usando la nueva columna, ¿cuáles son las 10 palabras más usadas? 

In [50]:
datos = pd.read_csv("dialogos.csv",encoding="utf-8")
datos

,Locución
0,Francamente no
1,"Lo del canadiense. Por favor, como que vosotro..."
2,Tienes razón. A lo mejor así te liberas de tu ...
3,"Pues, tú sabrás"
4,Para mí que fue Krieger
...,...
954,"Yo qué sé, digo yo qué será"
955,Lo han entendido mal. Las joyas que venden en ...
956,Oooo... Me llamo Vivian
957,¿De dónde? ¿Del coño de tu madre?


,Locución
0,Francamente no
1,"Lo del canadiense. Por favor, como que vosotro..."
2,Tienes razón. A lo mejor así te liberas de tu ...
3,"Pues, tú sabrás"
4,Para mí que fue Krieger
...,...
954,"Yo qué sé, digo yo qué será"
955,Lo han entendido mal. Las joyas que venden en ...
956,Oooo... Me llamo Vivian
957,¿De dónde? ¿Del coño de tu madre?


In [53]:
patron= "[^a-z-á-ÿ$ ]"
datos["pre-procesado"] = datos["Locución"].str.lower()
datos["pre-procesado"] = datos["pre-procesado"].replace(to_replace =patron, value = '', regex = True)
datos["pre-procesado"] = datos["pre-procesado"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_sp)]))
datos

,Locución,pre-procesado
0,Francamente no,francamente
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tiraríais allí si pudieseis
2,Tienes razón. A lo mejor así te liberas de tu ...,razón mejor así liberas energía nagativa
3,"Pues, tú sabrás",pues sabrás
4,Para mí que fue Krieger,krieger
...,...,...
954,"Yo qué sé, digo yo qué será",sé digo
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...
956,Oooo... Me llamo Vivian,oooo llamo vivian
957,¿De dónde? ¿Del coño de tu madre?,dónde coño madre


,Locución,pre-procesado
0,Francamente no,francamente
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tiraríais allí si pudieseis
2,Tienes razón. A lo mejor así te liberas de tu ...,razón mejor así liberas energía nagativa
3,"Pues, tú sabrás",pues sabrás
4,Para mí que fue Krieger,krieger
...,...,...
954,"Yo qué sé, digo yo qué será",sé digo
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...
956,Oooo... Me llamo Vivian,oooo llamo vivian
957,¿De dónde? ¿Del coño de tu madre?,dónde coño madre


In [80]:
oraciones= datos["pre-procesado"].to_numpy()
oraciones
freq={}
for oracion in oraciones:
    palabras = oracion.split(" ")
    for palabra in palabras:        
        freq[palabra]= (freq[palabra] +1) if palabra in freq else 1
firts10 = sorted(freq.items(), key=lambda x: x[1],reverse=True)    

vocabulario = [(k, v) for k, v in freq.items()]
firts10[1:11]    

[('bien', 36),
 ('si', 34),
 ('cómo', 34),
 ('así', 30),
 ('oh', 28),
 ('vamos', 26),
 ('sarcasmobol', 24),
 ('aquí', 24),
 ('bueno', 24),
 ('lana', 22)]

[('bien', 36),
 ('si', 34),
 ('cómo', 34),
 ('así', 30),
 ('oh', 28),
 ('vamos', 26),
 ('sarcasmobol', 24),
 ('aquí', 24),
 ('bueno', 24),
 ('lana', 22)]

# Punto 2: Representación vectorial

- `[15 pts]` Crear una bolsa de palabras (BoW) del corpus usando la columna pre-procesada (Usando la función de `sklearn`)
- `[10 pts]` ¿Cuántas palabras hay en el vocabulario? (Usando la función de `sklearn`)

,Locución,pre-procesado,bow
0,Francamente no,francamente,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tiraríais allí si pudieseis,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,Tienes razón. A lo mejor así te liberas de tu ...,razón mejor así liberas energía nagativa,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"Pues, tú sabrás",pues sabrás,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,Para mí que fue Krieger,krieger,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...
954,"Yo qué sé, digo yo qué será",sé digo,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
956,Oooo... Me llamo Vivian,oooo llamo vivian,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
957,¿De dónde? ¿Del coño de tu madre?,dónde coño madre,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [82]:
def bow(frase):
    frase_bow = [0]*len(vocabulario)
    for palabra,i in vocabulario:
        if palabra in frase.split():
            frase_bow[i] = 1
    return frase_bow

In [96]:
count_vect = CountVectorizer()
bow_rep = count_vect.fit_transform(datos["pre-procesado"].values)
#print("vacabulario :",count_vect.vocabulary_)
datos["bow"]=[row for row in bow_rep.toarray()]
datos

,Locución,pre-procesado,bow
0,Francamente no,francamente,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"Lo del canadiense. Por favor, como que vosotro...",canadiense favor tiraríais allí si pudieseis,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Tienes razón. A lo mejor así te liberas de tu ...,razón mejor así liberas energía nagativa,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"Pues, tú sabrás",pues sabrás,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Para mí que fue Krieger,krieger,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
954,"Yo qué sé, digo yo qué será",sé digo,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
955,Lo han entendido mal. Las joyas que venden en ...,entendido mal joyas venden canales siquiera in...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
956,Oooo... Me llamo Vivian,oooo llamo vivian,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
957,¿De dónde? ¿Del coño de tu madre?,dónde coño madre,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [94]:
len(count_vect.vocabulary_)

2115

2115

# Punto 3: 🤔

- `[3pts]` ¿Cuándo es una mejor idea usar una bolsa de n-gramas en vez de una bolsa de palabras?

In [98]:
print("Cuando las n palabras tiene un significado unico que por separado no seria lo mismo, como por ejemplo sapo perro, por separado son dos animales, pero juntas puede ser un insulto ")

Cuando las n palabras tiene un significado unico que por separado no seria lo mismo, como por ejemplo sapo perro, por separado son dos animales, pero juntas puede ser un insulto 


# BONUS Punto 4: 😜

- `[2pt]` ¿A qué pertenecen los dialogos de ese archivo? 

In [97]:
print("Se parece a un hilo de tweets, o a una conversación de un un grupo de personas, ya sea foro o algun canal de comunicación")

Se parece a un hilo de tweets, o a una conversación de un un grupo de fersonas, ya sea foro o algun canal de comunicación
